In [1]:
import pandas as pd
import requests
import json
import geopandas as gpd
from geopy.distance import distance #6

In [2]:
dataestimation = {'adresse': '10 Rue Barla 06300 Nice', 'housenumber': '10', 'coordinate': [7.282298, 43.701961], 'ville': 'Nice', 'citycode': '06088', 'codepostal': '06300', 'nbpièces': 4, 'nbsdb': 1, 'nbniveaux': 0, 'typedebien': 'Maison', 'surface': 85, 'surfaceterrain': 300, 'etatdubien': 'Standard', 'luminosité': 'Standard', 'calme': 'Standard', 'toiture': 'Standard', 'chauffage': '0'}

In [3]:
def APIEcoles(dataestimation, radius):  # radius 0.04
    lon = dataestimation["coordinate"][0];
    lat = dataestimation["coordinate"][1]
    url = f"https://data.education.gouv.fr/api/explore/v2.1/catalog/datasets/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/records?select=numero_uai%2Cappellation_officielle%2Csecteur_public_prive_libe%2Cadresse_uai%2Ccode_postal_uai%2Clibelle_commune%2Clatitude%2Clongitude%2Cnature_uai_libe%2Ccode_commune%2Clibelle_academie%2Cpatronyme_uai&where=longitude%20>%20{lon - radius}%20AND%20longitude%20<%20{lon + radius}%20AND%20latitude%20>%20{lat - radius}%20AND%20latitude%20<%20{lat + radius}&limit=100&offset=0"
    headers = {
        'Accept': 'application/json',
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        compteur = (response.json()["total_count"] - 1) // 100
        df = pd.DataFrame(response.json()["results"])
        if compteur > 0:
            for i in range(1, compteur + 1):
                url_ = f"https://data.education.gouv.fr/api/explore/v2.1/catalog/datasets/fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre/records?select=numero_uai%2Cappellation_officielle%2Csecteur_public_prive_libe%2Cadresse_uai%2Ccode_postal_uai%2Clibelle_commune%2Clatitude%2Clongitude%2Cnature_uai_libe%2Ccode_commune%2Clibelle_academie%2Cpatronyme_uai&where=longitude%20>%20{lon - radius}%20AND%20longitude%20<%20{lon + radius}%20AND%20latitude%20>%20{lat - radius}%20AND%20latitude%20<%20{lat + radius}&limit=100&offset={i * 100}"
                response_ = requests.get(url_, headers=headers)
                if response_.status_code == 200:
                    df_ = pd.DataFrame(response_.json()["results"])
                    df = pd.concat([df, df_])
                    df = df.reset_index(drop=True)

    # Mise en forme du dataframe
    df["adresse_uai"] = df["adresse_uai"] + ", " + df["code_postal_uai"] + " " + df["libelle_commune"]
    df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['longitude'], df['latitude']))
    df = df.drop(columns=['numero_uai', 'code_postal_uai', 'code_commune', 'patronyme_uai', 'latitude', 'longitude'])
    df['distance'] = round(df['geometry'].apply(lambda x: distance((x.x, x.y), dataestimation["coordinate"]).meters),
                           0).astype('int')  # Fonction pour calculer les distances
    df = df.sort_values("distance", ascending=True)

    # Mise à jour # Replace value in a list
    l = list(df["nature_uai_libe"])
    for i in range(len(l)):
        if l[i] == 'ECOLE DE NIVEAU ELEMENTAIRE':
            l[i] = 'ECOLE PRIMAIRE'
        if l[i] == 'ECOLE MATERNELLE':
            l[i] = 'ECOLE MATERNELLE'
        if l[i] == 'COLLEGE':
            l[i] = 'COLLEGE'
        if l[i] == 'LYCEE PROFESSIONNEL':
            l[i] = 'LYCEE'
        if l[i] == 'LYCEE ENSEIGNT GENERAL ET TECHNOLOGIQUE':
            l[i] = 'LYCEE'
        if l[i] == 'SECTION ENSEIGNT GEN. ET PROF. ADAPTE':
            l[i] = ''
        if l[i] == 'LYCEE POLYVALENT':
            l[i] = 'LYCEE'
        if l[i] == 'SECTION D ENSEIGNEMENT PROFESSIONNEL':
            l[i] = ''
        if l[i] == 'LYCEE D ENSEIGNEMENT GENERAL':
            l[i] = 'LYCEE'
        if l[i] == 'ECOLE DE NIVEAU ELEMENTAIRE SPECIALISEE':
            l[i] = ''
        if l[i] == 'SECTION ENSEIGT GENERAL ET TECHNOLOGIQUE':
            l[i] = ''
        if l[i] == 'LYCEE D ENSEIGNEMENT TECHNOLOGIQUE':
            l[i] = 'LYCEE'

    df["nature_uai_libe"] = l  # refaire le dataframe
    df = df[df["nature_uai_libe"] != '']  # supprimer les vides
    return df

In [4]:
df = APIEcoles(dataestimation, 0.04)

In [5]:
df.head()

,appellation_officielle,secteur_public_prive_libe,adresse_uai,libelle_commune,nature_uai_libe,libelle_academie,geometry,distance
178,Ecole primaire privée Saint Joseph,Privé,"14 rue Barla, 06300 Nice",Nice,ECOLE PRIMAIRE,Nice,POINT (7.28258 43.70187),33
19,Lycée général et technologique privé Saint Joseph,Privé,"14 RUE BARLA, 06300 Nice",Nice,LYCEE,Nice,POINT (7.28275 43.70176),55
169,Collège privé Saint Joseph,Privé,"14 RUE BARLA, 06300 Nice",Nice,COLLEGE,Nice,POINT (7.28292 43.70167),76
162,Ecole élémentaire Papon,Public,"6 rue Papon, 06300 Nice",Nice,ECOLE PRIMAIRE,Nice,POINT (7.28155 43.70159),93
128,Ecole maternelle Papon,Public,"4 rue Boyer, 06300 Nice",Nice,ECOLE MATERNELLE,Nice,POINT (7.28166 43.70132),100


In [17]:
%timeit geodictecole = df.to_geo_dict() # Utile pour store dans les dcc.store de dash

6.43 ms ± 254 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
%timeit gdf = gpd.GeoDataFrame.from_features(geodictecole) # manière de lire le dcc.store facilement

2.83 ms ± 250 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
